In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import statsmodels.api as sm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score
from sklearn.datasets import make_classification
from sklearn.metrics import r2_score
from scipy.optimize import minimize 
from scipy.optimize import nnls 
import pandas as pd
# NOTE: THIS CODE ONLY WORKS FOR UNI-DIMENSIONAL OUTPUT (REGRESSION OR CLASSIFICATION)

N = 1000
output = 'cls'  # 'cls' for classification 'proba' for probabilities, 'reg' for regression
k = 10


In [22]:
x, y = make_classification(n_samples=N, n_features=8, n_clusters_per_class=4,
                            n_informative=4, n_redundant=0, hypercube=False,
                            random_state=0, shuffle=True, class_sep=0.1)

def fn(x, A, b):
    return np.linalg.norm(A.dot(x) - b)

def combiner_solve(x, y):
    # adapted from https://stackoverflow.com/questions/33385898/how-to-include-constraint-to-scipy-nnls-function-solution-so-that-it-sums-to-1/33388181
    beta_0, rnorm = nnls(x,y)
    cons = {'type': 'eq', 'fun': lambda x:  np.sum(x)-1}
    bounds = [[0.0,None]]*x.shape[1]
    minout = minimize(fn, beta_0, args=(x, y), method='SLSQP',bounds=bounds,constraints=cons)
    beta = minout.x
    return beta


class SuperLearner(object):
    def __init__(self, output, est_dict, k, standardized_outcome=False):
        if standardized_outcome and (output=='proba' or output=='cls'):    
            print('WARNING: Standardizing outcome for a classification task will cause problems...')
        self.k = k  # number of cross validation folds
        self.beta = None
        self.output = output  # 'reg' for regression, 'proba' or 'cls' classification
        self.trained_superlearner = None
        self.est_dict = est_dict  # dictionary of learners/algos  
        self.standardized_outcome = standardized_outcome
        
        self.x_std = None
        self.x_mean = None
        self.y_std = None
        self.y_mean = None
    
    def fit(self, x, y):
        x = x.values if isinstance(x, pd.DataFrame) else x
        y = y.values[:,0] if isinstance(y, pd.DataFrame) else y
      
        # mean and std for full dataset (can be reused wth new data at prediction time)
        self.x_std = x.std(0)
        self.x_mean = x.mean(0)
        
        if self.standardized_outcome:
            self.y_std = y.std(0)
            self.y_mean = y.mean(0)

        if (self.output == 'cls') or (self.output == 'proba'):
            num_classes = np.unique(y)
            if len(num_classes) == 2:
                num_classes = 1
            elif len(num_classes) > 2:
                num_classes = len(num_classes)
                self.output = 'cat'
        else:
            num_classes = 1
            
        kf = KFold(n_splits=self.k, shuffle=True, random_state=0)
        
        all_preds = np.zeros((len(y), num_classes, len(self.est_dict)))  # for test preds
        
        i = 0
        for key in self.est_dict.keys():
            print('Training estimator:', key)
            
            est = self.est_dict[key]
            
            if self.output == 'proba' or (self.output == 'cls'):
                probs = []
            preds = []
            gts = []
            
            for train_index, test_index in kf.split(x):
                x_train = x[train_index]
                x_test =  x[test_index]
                y_train = y[train_index]
                y_test = y[test_index]

                # per train/test fold means and standard deviations
                x_std = x_train.std(0)
                x_mean = x_train.mean(0)
                x_train = (x_train - x_mean) / x_std
                x_test = (x_test - x_mean) / x_std
                
                if self.standardized_outcome:
                    y_std = y_train.std(0)
                    y_mean = y_train.mean(0)
                    y_train = (y_train - y_mean) / y_std
                    y_test = (y_test - y_mean) / y_std
                
                if key == 'poly':
                    est = LogisticRegression(C=1e2, max_iter=350) if (self.output == 'cls') or (
                                self.output == 'proba') else LinearRegression()
                    poly = PolynomialFeatures(2)
                    x_train_poly = poly.fit_transform(x_train)
                    x_test_poly = poly.fit_transform(x_test)

                    est.fit(x_train_poly, y_train)                 
                    
                else:
                    est.fit(x_train, y_train)
                    
                
                if self.output == 'proba' or (self.output == 'cls'):
                    p_robs = est.predict_proba(x_test_poly) if key == 'poly' else est.predict_proba(x_test)
                    probs.append(p_robs)

                p = est.predict(x_test_poly) if key == 'poly' else est.predict(x_test)
                preds.append(p)
                gts.append(y_test)
                
            preds = np.concatenate(preds)
            if self.output == 'proba' or (self.output == 'cls'):
                probs = np.concatenate(probs)
            if num_classes == 1:
                if self.output == 'proba' or (self.output == 'cls'):
                    probs = probs[:, 1].reshape(-1, 1)
                preds = preds.reshape(-1, 1)

            gts = np.concatenate(gts)
            
            if (self.output == 'cls') or (self.output == 'proba'):
                all_preds[:, :, i] = probs
            elif self.output == 'reg' or self.output == 'cat':
                all_preds[:, :, i] = preds
                
            i += 1
        
        # estimate betas on test predictions
        self.beta = combiner_solve(all_preds[:, 0, :], gts)  # all_preds is of shape [batch, categories, predictors]

        # now train each estimator on full dataset

        x = (x - self.x_mean) / self.x_std
        if self.standardized_outcome:
            y = (y-self.y_mean) / self.y_std
            
        for key in self.est_dict.keys():
            print('Training estimator on full data:', key)
            
            est = self.est_dict[key]
            
            if key == 'poly':
                est = LogisticRegression(C=1e2, max_iter=350) if (self.output == 'cls') or (
                            self.output == 'proba') else LinearRegression()
                poly = PolynomialFeatures(2)
                x_poly = poly.fit_transform(x)

                est.fit(x_poly, y)                    
                    
            else:
                est.fit(x, y)
                    
            self.est_dict[key] = est
            
            
    def predict(self, x):

        x_ = (x - self.x_mean) / self.x_std

        all_preds = np.zeros((len(x), len(self.est_dict)))
        i = 0

        for key in self.est_dict.keys():
            est = self.est_dict[key]
            if key == 'poly':
                poly = PolynomialFeatures(2)
                x_scaled = poly.fit_transform(x_)

            if (self.output == 'cls') or self.output == 'proba':
                preds = est.predict_proba(x_)[:, 1] if key != 'poly' else est.predict_proba(x_scaled)[:, 1]
            else:
                preds = est.predict(x_) if key != 'poly' else est.predict(x_scaled)
   

            all_preds[:, i] = preds

            i += 1
        
        weighted_preds = np.dot(all_preds, self.beta)
        weighted_preds = weighted_preds.reshape(-1, 1)
        if self.standardized_outcome:
            weighted_preds = (weighted_preds * self.y_std) + self.y_mean
        return weighted_preds

            

In [23]:
est_dict = {'LR':LogisticRegression()}

SL = SuperLearner(output='cls', est_dict=est_dict, k=k, standardized_outcome=False)
SL.fit(x, y)
preds = SL.predict(x)
preds = np.round(preds)
print(SL.beta, balanced_accuracy_score(y, preds))

Training estimator: LR
Training estimator on full data: LR
[1.] 0.552


In [24]:

est_dict = {'LR':LogisticRegression(), 'SVC':SVC(probability=True)}
SL = SuperLearner(output='cls', est_dict=est_dict, k=k, standardized_outcome=False)
SL.fit(x, y)
preds = SL.predict(x)
preds = np.round(preds)
print(SL.beta, balanced_accuracy_score(y, preds))


Training estimator: LR
Training estimator: SVC
Training estimator on full data: LR
Training estimator on full data: SVC
[8.10983225e-17 1.00000000e+00] 0.783


In [25]:

est_dict = {'LR':LogisticRegression(), 'SVC':SVC(probability=True), 'RF':RandomForestClassifier()}
SL = SuperLearner(output='cls', est_dict=est_dict, k=k, standardized_outcome=False)
SL.fit(x, y)
preds = SL.predict(x)
preds = np.round(preds)
print(SL.beta, balanced_accuracy_score(y, preds))


Training estimator: LR
Training estimator: SVC
Training estimator: RF
Training estimator on full data: LR
Training estimator on full data: SVC
Training estimator on full data: RF
[2.56895796e-16 8.19535772e-01 1.80464228e-01] 0.842
